In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.CodeMirror {font-family:Consolas; font-size:13pt;}
div.output {font-size:13pt; font-weight : bold;}
div.input {font-family:Consolas; font-size:13pt;}
</style>
"""))

In [85]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

### 한글원본 맞춤법 검사 ###
with open('Data/2_한글원본.txt', 'r', encoding='UTF8') as f:
    text = f.read()

ready_list = []

while len(text) > 500:
    temp_text = text[:500]
    last_space_idx = temp_text.rfind(' ')
    ready_list.append(text[:last_space_idx])
    
    text = text[last_space_idx: ]
    
ready_list.append(text)

dv = webdriver.Chrome('D:/Gray_Bigdata/src/07_Python/chromedriver.exe')
dv.get('http://www.naver.com')

elem = dv.find_element_by_name('query')
elem.clear()
elem.send_keys('네이버 맞춤법 검사기')
elem.send_keys(Keys.RETURN)

time.sleep(1)

textarea = dv.find_element_by_class_name('txt_gray')
results = ''
for ready in ready_list:
    textarea.send_keys(Keys.CONTROL, 'a')
    textarea.send_keys(ready)

    button = dv.find_element_by_class_name('btn_check')
    button.click()

    time.sleep(1)

    soup = BeautifulSoup(dv.page_source, 'html.parser')
    result = soup.select_one('p._result_text.stand_txt').text
    results += result + '\n\n'

time.sleep(2)
### 수정된 텍스트를 번역기 사이트를 통해 영어로 번역 ###
ready_list_eng = []

while len(results) > 300:
    temp_text = results[:300]
    last_space_idx = temp_text.rfind(' ')
    ready_list_eng.append(results[:last_space_idx])
    
    results = results[last_space_idx: ]

ready_list_eng.append(results)

dv.get('http://www.google.com')
elem = dv.find_element_by_name('q')
elem.clear()
elem.send_keys('구글 번역')
elem.send_keys(Keys.RETURN)

transBtn = dv.find_element_by_css_selector('span.tw-menu-btn-image')
transBtn.click()

time.sleep(1)

textarea_eng = dv.find_element_by_class_name('goog-textarea')

time.sleep(1)

results_eng = ''
for ready in ready_list_eng:    
    textarea_eng.send_keys(Keys.CONTROL, 'a')
    textarea_eng.send_keys(ready)
    
    time.sleep(1)
    
    soup = BeautifulSoup(dv.page_source, 'html.parser')
    result_eng= soup.select_one('div#tw-target-text-container pre#tw-target-text > span.Y2IQFc').text
    results_eng += result_eng + '\n'

with open('Data/2_자동화영어번역본.txt', 'w', encoding='UTF8') as f:
    f.write(results_eng)

time.sleep(1)
dv.close()